In [97]:

# Import libs
import pandas as pd
import numpy as np
from requests import get
from time import time
from time import sleep
from random import randint
from bs4 import BeautifulSoup
import dateparser
import re

from warnings import warn
from IPython.core.display import clear_output
import traceback

csv_filename='C:/Users/Robin/Dropbox/Travaux_&_Rapports_Stages/UbicomLab-GATech/MeToo/code/balancetonporc_com/stories_url.csv'

In [30]:
# Function to scrape the strories urls from http://www.balancetonporc.com/
def getUrl(nb_url, init_url):
    page1=get(init_url)
    url_list=['url story']
    for i in range(nb_url):
        soup = BeautifulSoup(page1.content, 'html.parser')
        url_box = soup.find('div',attrs={'class': 'prev-post'})
        url=url_box.a.get('href')
        url_list.append(url)
        page1=get(url)
        i+=1  
    # Saving the files
    
    r = np.asarray(url_list)
    np.savetxt("strories_url.csv", r, delimiter=",", fmt='%s')
            

In [89]:
getUrl(5,'http://www.balancetonporc.com/harcelement-sexuel-9/')

In [99]:
# Function to scrape the data from the movies urls
# The function return a dataframe and a list of url that return error.
# And save them into csv files (allocine_movies.csv and allocine_errors.csv)
def ScrapeURL(df_url):
    #### crer la liste des url
    
   # page1=get(init_url,)
   # url_list=[]
   # for i in range(nb_url):
   #     soup = BeautifulSoup(page1.content, 'html.parser')
   #     url_box = soup.find('div',attrs={'class': 'prev-post'})
   #     url=url_box.a.get('href')
   #     url_list.append(url)
   #     page1=get(url)
   #     i+=1 
    # init the dataframe
    c = ['title','author-name', 'published-date', 'nb-views','nb-comments','text','category', 'tags']
    df = pd.DataFrame(columns=c)
    
    # preparing the setting and monitoring loop
    start_time = time()
    n_request = 0
    
    # init list to save errors
    errors = []
    
    # request loop
    for i in range(len(df_url['url story'])):
        url=df_url['url story'].iloc[i]
        try :
            response = get(url)

            # Pause the loop
            sleep(randint(1,2))

            # Monitoring the requests
            n_request += 1
            
            elapsed_time = time() - start_time
            print('Request #{}; Frequency: {} requests/s'.format(n_request, n_request/elapsed_time))
            clear_output(wait = True)

            # Pause the loop
            sleep(randint(1,2))

            # Warning for non-200 status codes
            if response.status_code != 200:
                warn('Request #{}; Status code: {}'.format(n_request, response.status_code))
                errors.append(url)

            # Parse the content of the request with BeautifulSoup
            story_html_soup = BeautifulSoup(response.text, 'html.parser')

            if story_html_soup.find('title'):
                # Scrape the title
                tp_title = story_html_soup.find('title').text
                     

                # Set the defaut value
                tp_author_name = np.nan
                tp_published_date = np.nan
                tp_nb_views = np.nan
                tp_nb_comments= np.nan
                tp_text=np.nan
                tp_category=[]
                tp_tags = []
                tp_rank = np.nan
                

                #author name
                name=story_html_soup.find("span",attrs={'class': 'post-author-name'})
                tp_author_name=name.text.replace('Par','')
                               
                
                #published date
                tp_published_date= story_html_soup.find("meta",  property="article:published_time")['content']
                                
                #nbviews
                
                find_nb=re.compile(r'views post-meta-views rank-\d*')#the attribute of the class is changing from one page to another
                nb_text=find_nb.findall(str(response.text))[0]
                tp_nb_views=story_html_soup.find("span",attrs={'class':nb_text}).text
                                
                #nbcomments
                tp_nb_comments=story_html_soup.find("a",attrs={'class': 'comments'}).text
                
                #text
                story=story_html_soup.find("div",attrs={'class': 'entry-content clearfix single-post-content'})
                texte=''
                for pa in story.find_all('p'):
                    texte=texte+ pa.text
                tp_text=texte
                
                #category
                category= story_html_soup.find_all("meta",  property="article:section")
                for c in category:
                    tp_category.append(c['content'])
                
                #tags
                tag = story_html_soup.find_all("meta",  property="article:tag")
                for t in tag:
                    tp_tags.append(t['content'])
                
              
             
                # Append the data
                df_tmp = pd.DataFrame({'title': [tp_title],
                                       'category': [tp_category],
                                       'author-name': [tp_author_name],
                                       'published-date': [tp_published_date],
                                       'text': [tp_text],
                                       'tags': [tp_tags],
                                       'nb-views': [tp_nb_views],
                                       'nb-comments': [tp_nb_comments],
                                      'url': [url]})
                
                df = pd.concat([df, df_tmp], ignore_index=True)
        except:
            errors.append(url)
            warn('Request #{} fail; Total errors : {}'.format(n_request, len(errors)))
            traceback.print_exc()
            
    # monitoring 
    elapsed_time = time() - start_time
    print('Done; {} requests in {} seconds with {} errors'.format(n_request, round(elapsed_time, 0), len(errors)))
    clear_output(wait = True)
    df.to_json("strories_balancetonporc_com.json")
    r = np.asarray(errors)
    np.savetxt("strories_errors.csv", r, delimiter=",", fmt='%s')
    # return dataframe and errors
    return df, errors

In [103]:
# Load the list of urls 
s_url = pd.read_csv(csv_filename)


In [104]:
df_story,e=ScrapeURL(s_url)

Done; 1500 requests in 6091.0 seconds with 2 errors


In [106]:
df_story

,author-name,category,nb-comments,nb-views,published-date,tags,text,title,url
0,boni,[Au sein du couple],0,340,2017-11-21T20:16:33+01:00,[],"Humiliation, isolement, coups, insultes… Quand...",Mon calvaire en famille - Balance Ton Porc,http://www.balancetonporc.com/mon-calvaire-en-...
1,Louise75,[Autre],0,344,2017-11-21T19:55:52+01:00,[prédateur ; école ; voisin ; nounou ; enfance],J’ai grandi dans un petit village de Dordogne ...,#balanceTESporcs - Balance Ton Porc,http://www.balancetonporc.com/balancetesporcs/
2,chavant,[Cercle familial],1,187,2017-11-21T19:07:23+01:00,[],LETTRE A MES FILLES\nJe ne prends pas l’initia...,Pédophile - Balance Ton Porc,http://www.balancetonporc.com/pedophile-5/
3,Franchie,[Milieu médical],0,508,2017-11-21T18:50:19+01:00,"[anesthésie, Marseille, porc]",À l’âge de 17 ans pour ma première opération (...,Anesthésie pervers à Marseille - Balance Ton Porc,http://www.balancetonporc.com/anesthesie-marse...
4,Cactus,[Cercle familial],0,421,2017-11-21T17:55:21+01:00,[],Mon ex concubin qui a connu ma fille a l’âge d...,Agression sexuelle sur ma fille de 13 ans - Ba...,http://www.balancetonporc.com/agression-sexuel...
5,grangier,[Cercle familial],0,363,2017-11-21T17:31:49+01:00,[],J’ai 63 ans. J’ai été la victime d’un prédateu...,sale pervers - Balance Ton Porc,http://www.balancetonporc.com/sale-pervers/
6,Chippa,[Milieu médical],0,334,2017-11-21T17:15:16+01:00,"[médicin, Viol]",J’ai été violée par Docteur Hodeige de Couiza ...,Violée par Docteur Dominique Hodeige - Balance...,http://www.balancetonporc.com/violee-par-docte...
7,MM,[Autre],0,596,2017-11-21T16:35:32+01:00,"[abus, Attouchements, harcèlement moral..., ha...","J’avais 23 ans. Lui, un vieux monsieur, qui av...",Vieux pervers - Balance Ton Porc,http://www.balancetonporc.com/vieux-pervers/
8,Gwen,[Autre],0,627,2017-11-21T13:30:31+01:00,"[Rennes, vierge, Viol]","En 1999, j’ai été violée par un antillais d’un...",Viol pour ma 1ère relation sexuelle - Balance ...,http://www.balancetonporc.com/viol-pour-1ere-r...
9,A.,[Milieu scolaire],1,620,2017-11-21T13:15:39+01:00,"[College, Viol]",Ça se passait dans le vestiaires du gymnase du...,Violée à 13 ans - Balance Ton Porc,http://www.balancetonporc.com/violee-a-13-ans/


In [77]:
page = get("http://www.balancetonporc.com/pedocriminel-en-liberte-et-rien-ne-changera/")
page.content
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())
list(soup.children)
k=[type(item) for item in list(soup.children)]
#print(k)
html=list(soup.children)[1]
#####
#story_url=[]
#html_soup = BeautifulSoup(page.text, 'html.parser')
#stories = html_soup.find_all('section class')
#tag = soup.find_all("meta",  property="article:tag")
#tp_tag=[]
#for t in tag:
#    tp_tag.append(t['content'])
##print(tp_tag)
#name=name_box.a.get('href')
##print(name)
##for box in name_box:
#    #for link in box.find_all('a'):
#        #story_url.append(link.get('href'))
##print(story_url)
#title=html_soup.find('title').text
##print(title)
#tp_title = html_soup.find('div').text
##print(tp_title)
#print(tag)

text=str(page.content)
find_nb=re.compile(r'views post-meta-views rank-\d*')
nb_text=find_nb.findall(text)
nb=soup.find("span",attrs={'class':nb_text}).text

print(nb)



Accueil
